In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
import $ivy.`org.apache.spark::spark-mllib:2.4.0`
import $ivy.`sh.almond::ammonite-spark:0.4.0`
import $ivy.`org.datasyslab:geospark:1.2.0`

import org.apache.spark.serializer.KryoSerializer
import org.apache.spark.storage.StorageLevel
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.rdd.RDD
import org.datasyslab.geospark.enums.{GridType, IndexType}
import org.datasyslab.geospark.spatialOperator.JoinQuery
import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader
import scala.collection.JavaConverters._
import java.io._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

val spark = AmmoniteSparkSession.builder()
    .master("local[*]").appName("Validator")
    .getOrCreate()
import spark.implicits._
val appID = spark.sparkContext.applicationId

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                                    

import $ivy.$                                

import $ivy.$                              


import org.apache.spark.serializer.KryoSerializer

import org.apache.spark.storage.StorageLevel

import org.apache.spark.mllib.evaluation.RegressionMetrics

import org.apache.spark.rdd.RDD

import org.datasyslab.geospark.enums.{GridType, IndexType}

import org.datasyslab.geospark.spatialOperator.JoinQuery

import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader

import scala.collection.JavaConverters._

import java.io._

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@58bc7093
import spark.implicits._

appID: String = "local-1557884378542"

In [3]:
val df_phili_2000_path = "/home/acald013/RIDIR/Datasets/df_phili_2000.csv"
val df_phili_2000 = spark.read.option("header", "true").option("delimiter", "\t").csv(df_phili_2000_path)
df_phili_2000.show()

+-----------+-----------+----------------+----------------+
|      geoid|n_total_pop|n_white_under_15|p_white_under_15|
+-----------+-----------+----------------+----------------+
|42101000100|       2650|              34|            1.28|
|42101000200|       1362|               5|            0.37|
|42101000300|       2570|             102|            3.97|
|42101000400|       4313|              40|            0.93|
|42101000500|       1126|               1|            0.09|
|42101000600|       1315|               7|            0.53|
|42101000700|       2564|              32|            1.25|
|42101000800|       8461|             188|            2.22|
|42101000900|       4969|              50|            1.01|
|42101001000|       5808|             278|            4.79|
|42101001100|       5925|             124|            2.09|
|42101001200|       8148|             418|            5.13|
|42101001300|       4353|              75|            1.72|
|42101001400|       3585|              4

df_phili_2000_path: String = "/home/acald013/RIDIR/Datasets/df_phili_2000.csv"
df_phili_2000: DataFrame = [geoid: string, n_total_pop: string ... 2 more fields]

In [4]:
val df_phili_2010_path = "/home/acald013/RIDIR/Datasets/df_phili_2010.csv"
val df_phili_2010 = spark.read.option("header", "true").option("delimiter", "\t").csv(df_phili_2010_path)
df_phili_2010.show()

+-----------+-----------+----------------+------------------+
|      geoid|n_total_pop|n_white_under_15|  p_white_under_15|
+-----------+-----------+----------------+------------------+
|42101000100|     3183.0|            16.0|0.5026704366949418|
|42101000200|     2009.0|             0.0|               0.0|
|42101000300|     3509.0|           143.0| 4.075235109717868|
|42101000401|     2287.0|            20.0| 0.874508089199825|
|42101000402|     3021.0|            19.0| 0.628930817610063|
|42101000500|     2194.0|            13.0|0.5925250683682771|
|42101000600|     1565.0|             5.0|0.3194888178913738|
|42101000700|     2650.0|             0.0|               0.0|
|42101000801|     1713.0|            49.0| 2.860478692352598|
|42101000803|     3680.0|           134.0| 3.641304347826087|
|42101000804|     3224.0|            35.0|  1.08560794044665|
|42101000901|     1910.0|             0.0|               0.0|
|42101000902|     2367.0|            86.0| 3.633291085762569|
|4210100

df_phili_2010_path: String = "/home/acald013/RIDIR/Datasets/df_phili_2010.csv"
df_phili_2010: DataFrame = [geoid: string, n_total_pop: string ... 2 more fields]

In [5]:
val polys_1990_path = "/home/acald013/RIDIR/Datasets/polys_1990.csv"
val polys_1990 = spark.read.option("header", "true").option("delimiter", "\t").csv(polys_1990_path)
polys_1990.show()

+-----------+----+--------------------+
|      geoid|year|            geometry|
+-----------+----+--------------------+
|42101000001|1990|POLYGON ((-75.147...|
|42101000002|1990|POLYGON ((-75.159...|
|42101000003|1990|POLYGON ((-75.177...|
|42101000004|1990|POLYGON ((-75.178...|
|42101000005|1990|POLYGON ((-75.151...|
|42101000006|1990|POLYGON ((-75.153...|
|42101000007|1990|POLYGON ((-75.164...|
|42101000008|1990|POLYGON ((-75.178...|
|42101000009|1990|POLYGON ((-75.153...|
|42101000010|1990|POLYGON ((-75.144...|
|42101000011|1990|POLYGON ((-75.158...|
|42101000012|1990|POLYGON ((-75.180...|
|42101000013|1990|POLYGON ((-75.178...|
|42101000014|1990|POLYGON ((-75.175...|
|42101000015|1990|POLYGON ((-75.152...|
|42101000016|1990|POLYGON ((-75.144...|
|42101000017|1990|POLYGON ((-75.146...|
|42101000018|1990|POLYGON ((-75.153...|
|42101000019|1990|POLYGON ((-75.175...|
|42101000020|1990|POLYGON ((-75.180...|
+-----------+----+--------------------+
only showing top 20 rows



polys_1990_path: String = "/home/acald013/RIDIR/Datasets/polys_1990.csv"
polys_1990: DataFrame = [geoid: string, year: string ... 1 more field]

In [8]:
val polys_2000_path = "/home/acald013/RIDIR/Datasets/polys_2000.csv"
val polys_2000 = spark.read.option("header", "true").option("delimiter", "\t").csv(polys_2000_path)
polys_2000.show()

+-----------+----+--------------------+
|      geoid|year|            geometry|
+-----------+----+--------------------+
|42101000100|2000|POLYGON ((-75.147...|
|42101000200|2000|POLYGON ((-75.159...|
|42101000300|2000|POLYGON ((-75.177...|
|42101000400|2000|POLYGON ((-75.177...|
|42101000500|2000|POLYGON ((-75.152...|
|42101000600|2000|POLYGON ((-75.152...|
|42101000700|2000|POLYGON ((-75.163...|
|42101000800|2000|POLYGON ((-75.177...|
|42101000900|2000|POLYGON ((-75.152...|
|42101001000|2000|POLYGON ((-75.143...|
|42101001100|2000|POLYGON ((-75.157...|
|42101001200|2000|POLYGON ((-75.165...|
|42101001300|2000|POLYGON ((-75.177...|
|42101001400|2000|POLYGON ((-75.175...|
|42101001500|2000|POLYGON ((-75.152...|
|42101001600|2000|POLYGON ((-75.143...|
|42101001700|2000|POLYGON ((-75.146...|
|42101001800|2000|POLYGON ((-75.153...|
|42101001900|2000|POLYGON ((-75.174...|
|42101002000|2000|POLYGON ((-75.180...|
+-----------+----+--------------------+
only showing top 20 rows



polys_2000_path: String = "/home/acald013/RIDIR/Datasets/polys_2000.csv"
polys_2000: DataFrame = [geoid: string, year: string ... 1 more field]

In [9]:
val polys_2010_path = "/home/acald013/RIDIR/Datasets/polys_2010.csv"
val polys_2010 = spark.read.option("header", "true").option("delimiter", "\t").csv(polys_2010_path)
polys_2010.show()

+-----------+----+--------------------+
|      geoid|year|            geometry|
+-----------+----+--------------------+
|42101014500|2010|POLYGON ((-75.151...|
|42101031200|2010|POLYGON ((-75.084...|
|42101021800|2010|POLYGON ((-75.238...|
|42101009100|2010|POLYGON ((-75.202...|
|42101033600|2010|POLYGON ((-75.068...|
|42101004202|2010|POLYGON ((-75.156...|
|42101016800|2010|POLYGON ((-75.173...|
|42101007000|2010|POLYGON ((-75.226...|
|42101029100|2010|POLYGON ((-75.114...|
|42101980100|2010|POLYGON ((-75.232...|
|42101021300|2010|POLYGON ((-75.225...|
|42101030000|2010|POLYGON ((-75.085...|
|42101013300|2010|POLYGON ((-75.166...|
|42101012204|2010|POLYGON ((-75.220...|
|42101012203|2010|POLYGON ((-75.213...|
|42101013602|2010|POLYGON ((-75.185...|
|42101034502|2010|POLYGON ((-75.047...|
|42101000902|2010|POLYGON ((-75.160...|
|42101000804|2010|POLYGON ((-75.171...|
|42101001202|2010|POLYGON ((-75.174...|
+-----------+----+--------------------+
only showing top 20 rows



polys_2010_path: String = "/home/acald013/RIDIR/Datasets/polys_2010.csv"
polys_2010: DataFrame = [geoid: string, year: string ... 1 more field]

In [13]:
println(df_phili_2000.count())
println(df_phili_2010.count())
println()
println(polys_1990.count())
println(polys_2000.count())
println(polys_2010.count())


381
384

367
381
384


In [31]:
val phili_2000 = df_phili_2000.join(polys_2000, "geoid")
    .select("geometry", "geoid", "n_total_pop", "p_white_under_15")
phili_2000.orderBy("geoid").show
phili_2000.count()

+--------------------+-----------+-----------+----------------+
|            geometry|      geoid|n_total_pop|p_white_under_15|
+--------------------+-----------+-----------+----------------+
|POLYGON ((-75.147...|42101000100|       2650|            1.28|
|POLYGON ((-75.159...|42101000200|       1362|            0.37|
|POLYGON ((-75.177...|42101000300|       2570|            3.97|
|POLYGON ((-75.177...|42101000400|       4313|            0.93|
|POLYGON ((-75.152...|42101000500|       1126|            0.09|
|POLYGON ((-75.152...|42101000600|       1315|            0.53|
|POLYGON ((-75.163...|42101000700|       2564|            1.25|
|POLYGON ((-75.177...|42101000800|       8461|            2.22|
|POLYGON ((-75.152...|42101000900|       4969|            1.01|
|POLYGON ((-75.143...|42101001000|       5808|            4.79|
|POLYGON ((-75.157...|42101001100|       5925|            2.09|
|POLYGON ((-75.165...|42101001200|       8148|            5.13|
|POLYGON ((-75.177...|42101001300|      

phili_2000: DataFrame = [geometry: string, geoid: string ... 2 more fields]
res30_2: Long = 381L

In [32]:
val phili_2010 = df_phili_2010.join(polys_2010, "geoid")
    .select("geometry", "geoid", "n_total_pop", "p_white_under_15")
phili_2010.orderBy("geoid").show
phili_2010.count()

+--------------------+-----------+-----------+------------------+
|            geometry|      geoid|n_total_pop|  p_white_under_15|
+--------------------+-----------+-----------+------------------+
|POLYGON ((-75.152...|42101000100|     3183.0|0.5026704366949418|
|POLYGON ((-75.162...|42101000200|     2009.0|               0.0|
|POLYGON ((-75.179...|42101000300|     3509.0| 4.075235109717868|
|POLYGON ((-75.180...|42101000401|     2287.0| 0.874508089199825|
|POLYGON ((-75.173...|42101000402|     3021.0| 0.628930817610063|
|POLYGON ((-75.165...|42101000500|     2194.0|0.5925250683682771|
|POLYGON ((-75.164...|42101000600|     1565.0|0.3194888178913738|
|POLYGON ((-75.181...|42101000700|     2650.0|               0.0|
|POLYGON ((-75.183...|42101000801|     1713.0| 2.860478692352598|
|POLYGON ((-75.177...|42101000803|     3680.0| 3.641304347826087|
|POLYGON ((-75.171...|42101000804|     3224.0|  1.08560794044665|
|POLYGON ((-75.164...|42101000901|     1910.0|               0.0|
|POLYGON (

phili_2010: DataFrame = [geometry: string, geoid: string ... 2 more fields]
res31_2: Long = 384L

In [33]:
val phili_1990 = polys_1990.select("geometry", "geoid")
phili_1990.orderBy("geoid").show
phili_1990.count()

+--------------------+-----------+
|            geometry|      geoid|
+--------------------+-----------+
|POLYGON ((-75.147...|42101000001|
|POLYGON ((-75.159...|42101000002|
|POLYGON ((-75.177...|42101000003|
|POLYGON ((-75.178...|42101000004|
|POLYGON ((-75.151...|42101000005|
|POLYGON ((-75.153...|42101000006|
|POLYGON ((-75.164...|42101000007|
|POLYGON ((-75.178...|42101000008|
|POLYGON ((-75.153...|42101000009|
|POLYGON ((-75.144...|42101000010|
|POLYGON ((-75.158...|42101000011|
|POLYGON ((-75.180...|42101000012|
|POLYGON ((-75.178...|42101000013|
|POLYGON ((-75.175...|42101000014|
|POLYGON ((-75.152...|42101000015|
|POLYGON ((-75.144...|42101000016|
|POLYGON ((-75.146...|42101000017|
|POLYGON ((-75.153...|42101000018|
|POLYGON ((-75.175...|42101000019|
|POLYGON ((-75.180...|42101000020|
+--------------------+-----------+
only showing top 20 rows



phili_1990: DataFrame = [geometry: string, geoid: string]
res32_2: Long = 367L

In [38]:
def saveWKT(wkt: Dataset[String], filename: String): Unit = {
    val pw = new java.io.PrintWriter(filename)
    pw.write(wkt.collect().mkString(""))
    pw.close()
}

defined function saveWKT

In [40]:
saveWKT(phili_1990.map(p => s"${p.getString(0)}\t${p.getString(1)}\n"), "/home/acald013/RIDIR/Datasets/phili_1990.wkt")
saveWKT(phili_2000.map(p => s"${p.getString(0)}\t${p.getString(1)}\t${p.getString(2)}\t${p.getString(3)}\n"), "/home/acald013/RIDIR/Datasets/phili_2000.wkt")
saveWKT(phili_2010.map(p => s"${p.getString(0)}\t${p.getString(1)}\t${p.getString(2)}\t${p.getString(3)}\n"), "/home/acald013/RIDIR/Datasets/phili_2010.wkt")
